# OpenAI structured output with llama parse

tl;dr;

- much better in dealing with table
- still mixup the measurement a bit... perhaps make it less structured, i.e., ask what measurement it is and the units it use.
- Not extracting all information

In [1]:
import os
import nest_asyncio
from dotenv import load_dotenv
from openai import OpenAI

os.chdir("/workspaces/proto-carbonGPT")

from proto_carbongpt.data_model import Paper

load_dotenv()
nest_asyncio.apply()

In [20]:
# paper = "Knox et al., 2015"
# paper = "Rath et al., 2022"
# paper = "Villa et al., 2024"
paper = "Wang et al., 2022"

In [21]:
with open(f"data/markdowns/{paper}.md", "r") as f:
    text = f.read()

In [22]:
def extract(text: str) -> Paper:
    """Extract measurements from a text."""

    client = OpenAI()
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {
                "role": "system",
                "content": "Extract the research paper information. Get all the top soil organic carbon TOC measures in weights (e.g., g/kg, mg/g, mg/ha...) available in the paper. Also extract the changes in TOC measures over time if available.",
            },
            {"role": "user", "content": text},
        ],
        response_format=Paper,
    )
    return completion.choices[0].message.parsed

In [23]:
extracted = extract(text)

In [24]:
with open(f"data/parsed/{paper}.json", "w") as f:
    f.write(extracted.model_dump_json(indent=2))
